<a href="https://colab.research.google.com/github/aarushkachhawa/Deep-Fake-Detection/blob/main/DeepFakeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from skimage.io import imread
from skimage.color import convert_colorspace
from skimage import exposure
from skimage import feature
from skimage.color import rgb2gray
from skimage.transform import resize
import pandas as pd

import torch
from numba import cuda 
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2
import os
import tqdm

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import plot_model
from keras import callbacks

import gc

In [2]:
def preProcess(image):
    imgNewSpace = convert_colorspace(image, 'RGB', 'YCbCr')
    gammaEnhanced = exposure.adjust_gamma(imgNewSpace, 2)
    enhanced = convert_colorspace(gammaEnhanced, 'YCbCr','RGB')
    grayimg = rgb2gray(enhanced)
    edges = feature.canny(grayimg, sigma=3)
    return edges 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [4]:
def loadImages(dataPath,NoOfFilesPerClass):
    flat_data_arr=[] #input array
    target_arr=[] #output array
    for i in Categories:
        print(f'loading Category : {i}')
        path=os.path.join(datadir,i)
        for img in os.listdir(path)[:NoOfFilesPerClass]:
            img_array=imread(os.path.join(path,img))
            img_resized=resize(img_array,(150,150,3))
            imgPreprocess = preProcess(img_resized)
            flat_data_arr.append(imgPreprocess)
            target_arr.append(Categories.index(i))
        print(f'loaded category:{i} successfully')
    x=np.array(flat_data_arr)
    y=np.array(target_arr)
    return x,y

In [5]:
Categories=["real",'fake']

#path which contains all the categories of images
datadir='..140k_real_vs_fake/real_vs_fake/real-vs-fake/train/' 
NoOfFilesEachClass = 2000
x,y = loadImages(datadir,NoOfFilesEachClass)


loading Category : real


FileNotFoundError: ignored

In [ ]:
x.shape

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1254400   
                                                                 
 batch_normalization (BatchN  (None, 12544)            50176     
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 12544)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 7, 7, 128)        819200    
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 128)        5

In [ ]:
x = np.reshape(x, (x.shape[0], 150,150,1))

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')

tf.Tensor([[-0.00039641]], shape=(1, 1), dtype=float32)


In [ ]:
# device = cuda.get_current_device()
# device
torch.cuda.empty_cache()
gc.collect()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        1664      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from tensorflow.keras import backend as K

model = Sequential()

# Pamameters Initialization
inputShape = (x_train.shape[1], x_train.shape[2],x_train.shape[3])
activation = 'relu'
padding = 'same'
droprate = 0.1
epsilon=0.001

model = Sequential()
model.add(BatchNormalization(input_shape=inputShape))
model.add(Conv2D(filters=6, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))


model.add(Conv2D(filters=32, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=64, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(256, kernel_initializer='glorot_normal', activation='relu'))
          
model.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary() # Summary of the architecture

In [ ]:
from tensorflow.keras.optimizers import RMSprop
#opt = rmsprop(lr=0.0001, decay=1e-6)
#opt = adam(lr=0.001, decay=1e-6)
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, 
                    batch_size =150, 
                    epochs = 100, 
                    verbose = 1, 
#                     validation_data = (x_test, y_test),
                    validation_split =0.2,
                    shuffle=True,
                   )

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"])
plt.show()